# Library import

In [1]:
# 필요한 라이브러리를 임포트합니다.
import os
from typing import Tuple, Callable, Union, List
import random

import cv2
import timm
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models, transforms
from torchvision.transforms import AutoAugment, AutoAugmentPolicy
import albumentations as A
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from albumentations.pytorch import ToTensorV2
from PIL import Image
from torch.cuda.amp import GradScaler, autocast
import wandb
from wandb import Image as WandbImage
from sklearn.metrics import classification_report, confusion_matrix

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 랜덤 시드 설정 함수
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 시드 값 설정
seed = 42
set_seed(seed)


# Dataset Class

In [3]:
class CustomDataset(Dataset):
    def __init__(
        self, 
        root_dir: str, 
        info_df: pd.DataFrame, 
        transform: Callable,
        is_inference: bool = False
    ):
        self.root_dir = root_dir
        self.transform = transform
        self.is_inference = is_inference
        self.image_paths = info_df['image_path'].tolist()
        
        if not self.is_inference:
            self.targets = info_df['target'].tolist()

    def __len__(self) -> int:
        return len(self.image_paths)

    def __getitem__(self, index: int) -> Union[Tuple[torch.Tensor, int], torch.Tensor]:
        img_path = os.path.join(self.root_dir, self.image_paths[index])
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)
        if image is None:
            raise FileNotFoundError(f"Image not found: {img_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        image = self.transform(image)

        if self.is_inference:
            return image
        else:
            target = self.targets[index]
            return image, target

# Transform Class

In [4]:
class TorchvisionTransform:
    def __init__(self, is_train: bool = True):
        common_transforms = [
            transforms.Resize((448, 448)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
        
        if is_train:
            self.transform = transforms.Compose(
                [
                    # AutoAugment(policy=AutoAugmentPolicy.IMAGENET),
                    transforms.RandomHorizontalFlip(p=0.5),
                    transforms.RandomRotation(15),
                    transforms.ColorJitter(brightness=0.2, contrast=0.2)
                ] + common_transforms
            )
        else:
            self.transform = transforms.Compose(common_transforms)

    def __call__(self, image: Image.Image) -> torch.Tensor:
        return self.transform(image)

In [5]:
class AlbumentationsTransform:
    def __init__(self, is_train: bool = True):
        # 공통 변환 설정: 이미지 리사이즈, 정규화, 텐서 변환
        common_transforms = [
            A.Resize(448, 448),  # 이미지를 224x224 크기로 리사이즈
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 정규화
            ToTensorV2()  # albumentations에서 제공하는 PyTorch 텐서 변환
        ]
        
        if is_train:
            # 훈련용 변환: 랜덤 수평 뒤집기, 랜덤 회전, 랜덤 밝기 및 대비 조정 추가
            self.transform = A.Compose(
                [
                    A.HorizontalFlip(p=0.5),  # 50% 확률로 이미지를 수평 뒤집기
                    A.Rotate(limit=15),  # 최대 15도 회전
                    A.RandomBrightnessContrast(p=0.2),  # 밝기 및 대비 무작위 조정
                ] + common_transforms
            )
        else:
            # 검증/테스트용 변환: 공통 변환만 적용
            self.transform = A.Compose(common_transforms)

    def __call__(self, image) -> torch.Tensor:
        # 이미지가 NumPy 배열인지 확인
        if not isinstance(image, np.ndarray):
            raise TypeError("Image should be a NumPy array (OpenCV format).")
        
        # 이미지에 변환 적용 및 결과 반환
        transformed = self.transform(image=image)  # 이미지에 설정된 변환을 적용
        
        return transformed['image']  # 변환된 이미지의 텐서를 반환

In [6]:
class TransformSelector:
    """
    이미지 변환 라이브러리를 선택하기 위한 클래스.
    """
    def __init__(self, transform_type: str):

        # 지원하는 변환 라이브러리인지 확인
        if transform_type in ["torchvision", "albumentations"]:
            self.transform_type = transform_type
        
        else:
            raise ValueError("Unknown transformation library specified.")

    def get_transform(self, is_train: bool):
        
        # 선택된 라이브러리에 따라 적절한 변환 객체를 생성
        if self.transform_type == 'torchvision':
            transform = TorchvisionTransform(is_train=is_train)
        
        elif self.transform_type == 'albumentations':
            transform = AlbumentationsTransform(is_train=is_train)
        
        return transform

# Model Class

In [7]:
class SimpleCNN(nn.Module):
    """
    간단한 CNN 아키텍처를 정의하는 클래스.
    """
    def __init__(self, num_classes: int):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        # 순전파 함수 정의
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [8]:
class TorchvisionModel(nn.Module):
    """
    Torchvision에서 제공하는 사전 훈련된 모델을 사용하는 클래스.
    """
    def __init__(
        self, 
        model_name: str, 
        num_classes: int, 
        pretrained: bool
    ):
        super(TorchvisionModel, self).__init__()
        self.model = models.__dict__[model_name](pretrained=pretrained)
        
        # 모델의 최종 분류기 부분을 사용자 정의 클래스 수에 맞게 조정
        if 'fc' in dir(self.model):
            num_ftrs = self.model.fc.in_features
            self.model.fc = nn.Linear(num_ftrs, num_classes)
        
        elif 'classifier' in dir(self.model):
            num_ftrs = self.model.classifier[-1].in_features
            self.model.classifier[-1] = nn.Linear(num_ftrs, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        return self.model(x)

In [9]:
class TimmModel(nn.Module):
    """
    Timm 라이브러리를 사용하여 다양한 사전 훈련된 모델을 제공하는 클래스.
    """
    def __init__(
        self, 
        model_name: str, 
        num_classes: int, 
        pretrained: bool
    ):
        super(TimmModel, self).__init__()
        self.model = timm.create_model(
            model_name, 
            pretrained=pretrained, 
            num_classes=num_classes
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        return self.model(x)

In [10]:
class ModelSelector:
    """
    사용할 모델 유형을 선택하는 클래스.
    """
    def __init__(
        self, 
        model_type: str, 
        num_classes: int, 
        **kwargs
    ):
        
        # 모델 유형에 따라 적절한 모델 객체를 생성
        if model_type == 'simple':
            self.model = SimpleCNN(num_classes=num_classes)
        
        elif model_type == 'torchvision':
            self.model = TorchvisionModel(num_classes=num_classes, **kwargs)
        
        elif model_type == 'timm':
            self.model = TimmModel(num_classes=num_classes, **kwargs)
        
        else:
            raise ValueError("Unknown model type specified.")

    def get_model(self) -> nn.Module:

        # 생성된 모델 객체 반환
        return self.model

# Loss Class

In [11]:
class Loss(nn.Module):
    """
    모델의 손실함수를 계산하는 클래스.
    """
    def __init__(self):
        super(Loss, self).__init__()
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(
        self, 
        outputs: torch.Tensor, 
        targets: torch.Tensor
    ) -> torch.Tensor:
    
        return self.loss_fn(outputs, targets)

# Trainer Class

In [12]:
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0):
        """
        Args:
            patience (int): 개선이 없을 때 몇 에포크를 기다릴지
            min_delta (float): 성능이 개선되었다고 판단하는 최소 변화량
        """
        self.patience = patience  # 개선되지 않아도 기다리는 최대 에포크 수
        self.min_delta = min_delta  # 성능 개선이 없다고 판단하는 최소 변화량
        self.counter = 0  # 개선되지 않은 에포크 수 카운트
        self.best_loss = None  # 검증 손실의 최저값
        self.early_stop = False  # 중지 플래그
    
    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss  # 첫 번째 에포크의 손실 저장
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss  # 손실이 개선되면 갱신
            self.counter = 0  # 카운터 초기화
        else:
            self.counter += 1  # 손실이 개선되지 않으면 카운터 증가
            if self.counter >= self.patience:
                self.early_stop = True  # patience를 초과하면 학습 중지

In [13]:
class Trainer:
    def __init__(
        self, 
        model: nn.Module, 
        device: torch.device, 
        train_loader: DataLoader, 
        val_loader: DataLoader, 
        optimizer: optim.Optimizer,
        scheduler: optim.lr_scheduler._LRScheduler,
        loss_fn: torch.nn.modules.loss._Loss, 
        epochs: int,
        result_path: str,
        num_classes: int,  # num_classes 추가
        patience: int = 5,  # Early Stopping patience 추가
        min_delta: float = 0.01  # Early Stopping min_delta 추가
    ):
        # 클래스 초기화: 모델, 디바이스, 데이터 로더 등 설정
        self.model = model  # 훈련할 모델
        self.device = device  # 연산을 수행할 디바이스 (CPU or GPU)
        self.train_loader = train_loader  # 훈련 데이터 로더
        self.val_loader = val_loader  # 검증 데이터 로더
        self.optimizer = optimizer  # 최적화 알고리즘
        self.scheduler = scheduler  # 학습률 스케줄러
        self.loss_fn = loss_fn  # 손실 함수
        self.epochs = epochs  # 총 훈련 에폭 수
        self.result_path = result_path  # 모델 저장 경로
        self.best_models = []  # 가장 좋은 상위 3개 모델의 정보를 저장할 리스트
        self.lowest_loss = float('inf')  # 가장 낮은 Loss를 저장할 변수
        self.early_stopping = EarlyStopping(patience=patience, min_delta=min_delta)  # EarlyStopping 초기화
        self.num_classes = num_classes  # num_classes 추가
        # global_step 변수 초기화
        self.global_step = 0  # 배치 수를 추적하기 위한 변수
        # wandb watch
        wandb.watch(self.model, log='all')

    def save_model(self, epoch, loss):
        # 모델 저장 경로 설정
        os.makedirs(self.result_path, exist_ok=True)

        # 현재 에폭 모델 저장
        current_model_path = os.path.join(self.result_path, f'model_epoch_{epoch}_loss_{loss:.4f}.pt')
        torch.save(self.model.state_dict(), current_model_path)

        # 최상위 3개 모델 관리
        self.best_models.append((loss, epoch, current_model_path))
        self.best_models.sort()
        if len(self.best_models) > 3:
            _, _, path_to_remove = self.best_models.pop(-1)  # 가장 높은 손실 모델 삭제
            if os.path.exists(path_to_remove):
                os.remove(path_to_remove)

        # 가장 낮은 손실의 모델 저장
        if loss < self.lowest_loss:
            self.lowest_loss = loss
            best_model_path = os.path.join(self.result_path, 'best_model.pt')
            torch.save(self.model.state_dict(), best_model_path)
            print(f"Save {epoch}epoch result. Loss = {loss:.4f}")

    def train_epoch(self, epoch) -> tuple:
        # 한 에폭 동안의 훈련을 진행
        self.model.train()
        
        total_loss = 0.0
        correct = 0
        total = 0
        progress_bar = tqdm(self.train_loader, desc="Training", leave=False)
        scaler = GradScaler()  # AMP를 위한 GradScaler 객체 생성

        for images, targets in progress_bar:
            images, targets = images.to(self.device), targets.to(self.device)
            self.optimizer.zero_grad()

            # autocast 컨텍스트 내에서 모델을 실행하여 정밀도를 관리
            with autocast():
                outputs = self.model(images)
                loss = self.loss_fn(outputs, targets)

            # 스케일링된 손실을 사용하여 역전파 실행
            scaler.scale(loss).backward()

            # 스케일러를 사용해 가중치를 업데이트
            scaler.step(self.optimizer)
            scaler.update()

            total_loss += loss.item()

            # 정확도 계산
            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()
            total += targets.size(0)

            progress_bar.set_postfix(loss=loss.item())

            # wandb에 배치별 손실 및 정확도 로깅
            self.global_step += 1
            batch_accuracy = 100.0 * predicted.eq(targets).sum().item() / targets.size(0)
            wandb.log({
                'train_loss_batch': loss.item(),
                'train_accuracy_batch': batch_accuracy,
                'global_step': self.global_step
            })

        # 전체 정확도 계산
        train_accuracy = 100.0 * correct / total

        # Log metrics to wandb
        wandb.log({
            'train_loss_epoch': total_loss / len(self.train_loader),
            'train_accuracy_epoch': train_accuracy,
            'epoch': epoch+1
        })

        return total_loss / len(self.train_loader), train_accuracy

    def validate(self, epoch) -> tuple:
        # 모델의 검증을 진행
        self.model.eval()
        
        total_loss = 0.0
        correct = 0
        total = 0
        true_labels = []
        pred_labels = []
        misclassified_images = []
        class_total = [0] * self.num_classes
        class_correct = [0] * self.num_classes
        class_misclassified = [0] * self.num_classes  # 클래스별 오분류 개수
        progress_bar = tqdm(self.val_loader, desc="Validating", leave=False)
        
        max_images_to_log = 20  # 로그할 최대 이미지 수
        num_logged_images = 0
        
        with torch.no_grad():
            for images, targets in progress_bar:
                images, targets = images.to(self.device), targets.to(self.device)
                outputs = self.model(images)
                loss = self.loss_fn(outputs, targets)
                total_loss += loss.item()

                # 정확도 계산
                _, predicted = outputs.max(1)
                correct_tensor = predicted.eq(targets)
                correct += correct_tensor.sum().item()
                total += targets.size(0)

                true_labels.extend(targets.cpu().numpy())
                pred_labels.extend(predicted.cpu().numpy())

                progress_bar.set_postfix(loss=loss.item())

                # wandb에 배치별 검증 손실 로깅
                wandb.log({
                    'val_loss_batch': loss.item(),
                    'global_step': self.global_step
                })

                # 클래스별 정답 및 오답 개수 집계
                for t, p in zip(targets.view(-1), predicted.view(-1)):
                    class_total[t.item()] += 1
                    if t == p:
                        class_correct[t.item()] += 1
                    else:
                        class_misclassified[t.item()] += 1

                # 잘못 분류된 이미지 추출
                mis_indices = (predicted != targets).nonzero(as_tuple=False).squeeze()
                if mis_indices.numel() > 0 and num_logged_images < max_images_to_log:
                    mis_images = images[mis_indices]
                    mis_targets = targets[mis_indices]
                    mis_preds = predicted[mis_indices]

                    # 이미지 로그 (최대 max_images_to_log개)
                    for img, true_label, pred_label in zip(mis_images, mis_targets, mis_preds):
                        if num_logged_images >= max_images_to_log:
                            break
                        img = img.cpu()
                        img_np = img.permute(1, 2, 0).numpy()
                        # 이미지 정규화 해제
                        img_np = img_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
                        img_np = np.clip(img_np, 0, 1)
                        caption = f"True: {true_label.item()}, Pred: {pred_label.item()}"
                        misclassified_images.append(wandb.Image(img_np, caption=caption))
                        num_logged_images += 1

        # 잘못 분류된 이미지들을 Wandb에 로그
        wandb.log({"misclassified_images": misclassified_images, 'epoch': epoch+1})

        # 전체 정확도 계산
        val_accuracy = 100.0 * correct / total

        # 클래스별 정확도 및 오분류 수 계산
        per_class_accuracy = {}
        per_class_misclassifications = {}
        for i in range(self.num_classes):
            if class_total[i] > 0:
                accuracy = 100.0 * class_correct[i] / class_total[i]
                misclassifications = class_misclassified[i]
            else:
                accuracy = None
                misclassifications = None
            per_class_accuracy[i] = accuracy
            per_class_misclassifications[i] = misclassifications

        # 클래스별 오분류 수를 wandb에 로그
        wandb.log({
            'per_class_misclassifications': wandb.Histogram(list(per_class_misclassifications.values())),
            'epoch': epoch+1
        })

        # 가장 많이 틀린 클래스 상위 5개 찾기
        sorted_misclassifications = sorted(per_class_misclassifications.items(), key=lambda x: x[1] if x[1] is not None else -1, reverse=True)
        worst_classes = sorted_misclassifications[:5]

        # 잘못된 예측이 많은 클래스 로그
        worst_classes_table = wandb.Table(columns=["class_id", "misclassifications"])
        for class_id, miscount in worst_classes:
            worst_classes_table.add_data(class_id, miscount)
        wandb.log({"worst_classes_misclassifications": worst_classes_table})

        # Log metrics to wandb
        wandb.log({
            'val_loss_epoch': total_loss / len(self.val_loader),
            'val_accuracy_epoch': val_accuracy,
            'epoch': epoch+1,
        })

        # Log confusion matrix to wandb
        wandb.log({"confusion_matrix": wandb.plot.confusion_matrix(probs=None,
                                                                   y_true=true_labels,
                                                                   preds=pred_labels,
                                                                   class_names=[str(i) for i in range(self.num_classes)])})

        return total_loss / len(self.val_loader), val_accuracy


    def train(self) -> None:
        # 전체 훈련 과정을 관리
        for epoch in range(self.epochs):
            print(f"Epoch {epoch+1}/{self.epochs}")

            train_loss, train_accuracy = self.train_epoch(epoch)
            val_loss, val_accuracy = self.validate(epoch)

            print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")
            print(f"Epoch {epoch+1}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%\n")

            self.save_model(epoch, val_loss)

            # Early Stopping 조건 확인
            self.early_stopping(val_loss)
            if self.early_stopping.early_stop:
                print("Early stopping triggered. Stopping training...")
                break

            self.scheduler.step()

# Model Training

In [14]:
# 학습에 사용할 장비를 선택.
# torch라이브러리에서 gpu를 인식할 경우, cuda로 설정.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
# 학습 데이터의 경로와 정보를 가진 파일의 경로를 설정.
traindata_dir = "/data/ephemeral/home/data/train"
traindata_info_file = "/data/ephemeral/home/data/train.csv"
save_result_path = "/data/ephemeral/home/youngtae/model2/Autoaugment_training_result"

In [16]:
# 학습 데이터의 class, image path, target에 대한 정보가 들어있는 csv파일을 읽기.
train_info = pd.read_csv(traindata_info_file)

# 총 class의 수를 측정.
num_classes = len(train_info['target'].unique())

# 각 class별로 8:2의 비율이 되도록 학습과 검증 데이터를 분리.
train_df, val_df = train_test_split(
    train_info, 
    test_size=0.2,
    stratify=train_info['target'],
    random_state=seed  # 랜덤 시드 적용
)

In [17]:
# Transform 설정
transform_selector = TransformSelector(
    transform_type="torchvision"
)
train_transform = transform_selector.get_transform(is_train=True)
val_transform = transform_selector.get_transform(is_train=False)

# Dataset 생성
train_dataset = CustomDataset(
    root_dir=traindata_dir,
    info_df=train_df,
    transform=train_transform
)
val_dataset = CustomDataset(
    root_dir=traindata_dir,
    info_df=val_df,
    transform=val_transform
)

In [18]:
# DataLoader에서 worker_init_fn을 설정하여 각 워커의 시드를 고정
def worker_init_fn(worker_id):
    np.random.seed(seed + worker_id)
    random.seed(seed + worker_id)

# DataLoader 생성
train_loader = DataLoader(
    train_dataset, 
    batch_size=64, 
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    worker_init_fn=worker_init_fn
)
val_loader = DataLoader(
    val_dataset, 
    batch_size=64, 
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    worker_init_fn=worker_init_fn
)

In [19]:
# wandb 초기화
wandb.init(project='experiment_baseline_2', config={
    'epochs': 50,
    'batch_size': 64,
    'learning_rate': 0.001,
    'optimizer': 'AdamW',
    'model': 'eva02_large_patch14_448.mim_m38m_ft_in22k_in1k',
    'transform': 'AutoAugment',
    'seed': seed,
    'num_classes': num_classes
    # Add other hyperparameters here
})

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [20]:
# 학습에 사용할 Model을 선언.
model_selector = ModelSelector(
    model_type='timm', 
    num_classes=num_classes,
    model_name='eva02_large_patch14_448.mim_m38m_ft_in22k_in1k', 
    pretrained=True
)
model = model_selector.get_model()

# 선언된 모델을 학습에 사용할 장비로 셋팅.
model.to(device)

# 모델의 모든 파라미터를 동결 (학습되지 않도록 설정)
for name, param in model.named_parameters():
    param.requires_grad = False

# block.22, block.23과 head 레이어만 학습되도록 설정
for name, param in model.named_parameters():
    if 'blocks.23' in name or 'head' in name:
        param.requires_grad = True


In [21]:
# optimizer 설정
optimizer = optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()), 
    lr=wandb.config.learning_rate  # wandb.config에서 학습률 사용
)

In [22]:
# 스케줄러 초기화
scheduler_step_size = 30  # 매 30step마다 학습률 감소
scheduler_gamma = 0.1  # 학습률을 현재의 10%로 감소

# 한 epoch당 step 수 계산
steps_per_epoch = len(train_loader)

# 2 epoch마다 학습률을 감소시키는 스케줄러 선언
epochs_per_lr_decay = 2
scheduler_step_size = steps_per_epoch * epochs_per_lr_decay

scheduler = optim.lr_scheduler.StepLR(
    optimizer, 
    step_size=scheduler_step_size, 
    gamma=scheduler_gamma
)

In [23]:
# 학습에 사용할 Loss를 선언.
loss_fn = Loss()

In [24]:
# 앞서 선언한 필요 class와 변수들을 조합해, 학습을 진행할 Trainer를 선언.
trainer = Trainer(
    model=model, 
    device=device, 
    train_loader=train_loader,
    val_loader=val_loader, 
    optimizer=optimizer,
    scheduler=scheduler,
    loss_fn=loss_fn, 
    epochs=wandb.config.epochs,
    result_path=save_result_path,
    num_classes=num_classes  # 추가됨
)

In [25]:
# 모델 학습.
trainer.train()

Epoch 1/50


Epoch 1, Train Loss: 1.1720, Train Accuracy: 77.62%
Epoch 1, Validation Loss: 0.4688, Validation Accuracy: 87.35%

Save 0epoch result. Loss = 0.4688
Epoch 2/50


Epoch 2, Train Loss: 0.3335, Train Accuracy: 90.67%
Epoch 2, Validation Loss: 0.4129, Validation Accuracy: 88.45%

Save 1epoch result. Loss = 0.4129
Epoch 3/50


Epoch 3, Train Loss: 0.2425, Train Accuracy: 93.03%
Epoch 3, Validation Loss: 0.3845, Validation Accuracy: 89.92%

Save 2epoch result. Loss = 0.3845
Epoch 4/50


Epoch 4, Train Loss: 0.1753, Train Accuracy: 94.62%
Epoch 4, Validation Loss: 0.4428, Validation Accuracy: 89.88%

Epoch 5/50


Epoch 5, Train Loss: 0.1598, Train Accuracy: 95.31%
Epoch 5, Validation Loss: 0.4429, Validation Accuracy: 89.52%

Epoch 6/50


Epoch 6, Train Loss: 0.1428, Train Accuracy: 95.91%
Epoch 6, Validation Loss: 0.4095, Validation Accuracy: 90.55%

Epoch 7/50


Epoch 7, Train Loss: 0.1141, Train Accuracy: 96.63%
Epoch 7, Validation Loss: 0.4395, Validation Accuracy: 90.22%

Epoch 8/50


Epoch 8, Train Loss: 0.1154, Train Accuracy: 96.48%
Epoch 8, Validation Loss: 0.4396, Validation Accuracy: 90.28%

Early stopping triggered. Stopping training...


In [26]:
# wandb run 종료
wandb.finish()

epoch,▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇████
global_step,▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇█████
train_accuracy_batch,▁▅▅▅▆▆▇▇▇▇▇▇▇▆█▆▇▇█▇▇█▇▇██▇▇▇▇█▇███▇▇█▇▇
train_accuracy_epoch,▁▆▇▇████
train_loss_batch,█▆▂▂▁▁▂▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_epoch,█▂▂▁▁▁▁▁
val_accuracy_epoch,▁▃▇▇▆█▇▇
val_loss_batch,▅█▄▆▂▃▆▆▆▃▃▅▅▅▄▃▇▄▃▅▃▅▅▅▂▂▆▄▄▂█▃▄▁▆▇▆▆▂▄
val_loss_epoch,█▃▁▆▆▃▆▆
epoch,8
global_step,1504


# Inference

In [27]:
# 모델 추론을 위한 함수
def inference(
    model: nn.Module, 
    device: torch.device, 
    test_loader: DataLoader
):
    # 모델을 평가 모드로 설정
    model.to(device)
    model.eval()
    
    predictions = []
    with torch.no_grad():  # Gradient 계산을 비활성화
        for images in tqdm(test_loader):
            # 데이터를 같은 장치로 이동
            images = images.to(device)
            
            # 모델을 통해 예측 수행
            logits = model(images)
            logits = F.softmax(logits, dim=1)
            preds = logits.argmax(dim=1)
            
            # 예측 결과 저장
            predictions.extend(preds.cpu().detach().numpy())  # 결과를 CPU로 옮기고 리스트에 추가
    
    return predictions

In [28]:
# 추론 데이터의 경로와 정보를 가진 파일의 경로를 설정.
testdata_dir = "/data/ephemeral/home/data/test"
testdata_info_file = "/data/ephemeral/home/data/test.csv"
save_result_path = "/data/ephemeral/home/youngtae/model2/Autoaugment_training_result"

In [29]:
# 추론 데이터의 class, image path, target에 대한 정보가 들어있는 csv파일을 읽기.
test_info = pd.read_csv(testdata_info_file)

# 총 class 수.
num_classes = 500

In [30]:
# 추론에 사용할 Transform을 선언.
transform_selector = TransformSelector(
    transform_type = "torchvision"
)
test_transform = transform_selector.get_transform(is_train=False)

# 추론에 사용할 Dataset을 선언.
test_dataset = CustomDataset(
    root_dir=testdata_dir,
    info_df=test_info,
    transform=test_transform,
    is_inference=True
)

# 추론에 사용할 DataLoader를 선언.
test_loader = DataLoader(
    test_dataset, 
    batch_size=64, 
    shuffle=False,
    drop_last=False,
    num_workers=4,
    pin_memory=True,
    worker_init_fn=worker_init_fn
)

In [31]:
# 추론에 사용할 장비를 선택.
# torch라이브러리에서 gpu를 인식할 경우, cuda로 설정.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 추론에 사용할 Model을 선언.
model_selector = ModelSelector(
    model_type='timm', 
    num_classes=num_classes,
    model_name='eva02_large_patch14_448.mim_m38m_ft_in22k_in1k', 
    pretrained=False
)
model = model_selector.get_model()


In [32]:
# best epoch 모델을 불러오기.
model.load_state_dict(
    torch.load(
        os.path.join(save_result_path, "best_model.pt"),
        map_location=device
    )
)

<All keys matched successfully>

In [33]:
# predictions를 CSV에 저장할 때 형식을 맞춰서 저장
# 테스트 함수 호출
predictions = inference(
    model=model, 
    device=device, 
    test_loader=test_loader
)

100%|██████████| 157/157 [10:32<00:00,  4.03s/it]


In [34]:
# 모든 클래스에 대한 예측 결과를 하나의 문자열로 합침
test_info['target'] = predictions
test_info = test_info.reset_index().rename(columns={"index": "ID"})
test_info

,ID,image_path,target
0,0,0.JPEG,328
1,1,1.JPEG,414
2,2,2.JPEG,493
3,3,3.JPEG,17
4,4,4.JPEG,388
...,...,...,...
10009,10009,10009.JPEG,235
10010,10010,10010.JPEG,191
10011,10011,10011.JPEG,466
10012,10012,10012.JPEG,258


In [35]:
# DataFrame 저장
test_info.to_csv("output.csv", index=False)